In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Load the Pima Indians Diabetes dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
df = pd.read_csv(url, names=columns)

# Split the dataset into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Normalize the feature data using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)  # Make it a column vector
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)


In [ ]:
from sklearn.preprocessing import StandardScaler

diabetes_data_copy = diabetes_df.copy(deep = True)

sc_X = StandardScaler()
X = pd.DataFrame(sc_X.fit_transform(diabetes_data_copy.drop(["Outcome"],axis = 1)),
        columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
                 'BMI', 'DiabetesPedigreeFunction', 'Age'
                 ])
X.head()

In [ ]:
class MLPModel(nn.Module):
    def __init__(self):
        super(MLPModel, self).__init__()

        # Input layer: 8 features (input size) -> Hidden Layer 1: 64 neurons
        self.fc1 = nn.Linear(8, 64)
        self.fc2 = nn.Linear(64, 32)  # Hidden Layer 2: 32 neurons
        self.fc3 = nn.Linear(32, 1)   # Output layer: 1 neuron for binary classification

        # Sigmoid activation for binary classification
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Apply ReLU activation to fc1
        x = torch.softmax(self.fc2(x))  # Apply ReLU activation to fc2
        x = self.fc3(x)              # No activation function here (sigmoid applied later)
        x = self.sigmoid(x)          # Sigmoid activation for binary output
        return x


In [ ]:
# Initialize the MLP model, loss function, and optimizer
model = MLPModel()
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 1500
for epoch in range(epochs):
    model.train()  # Set the model to training mode

    optimizer.zero_grad()  # Zero the gradients

    # Forward pass
    outputs = model(X_train)

    # Compute the loss
    loss = criterion(outputs, y_train)

    # Backward pass
    loss.backward()

    # Update the weights
    optimizer.step()

    # Print the loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [10/1500], Loss: 0.6374
Epoch [20/1500], Loss: 0.5956
Epoch [30/1500], Loss: 0.5527
Epoch [40/1500], Loss: 0.5138
Epoch [50/1500], Loss: 0.4836
Epoch [60/1500], Loss: 0.4624
Epoch [70/1500], Loss: 0.4482
Epoch [80/1500], Loss: 0.4386
Epoch [90/1500], Loss: 0.4308
Epoch [100/1500], Loss: 0.4236
Epoch [110/1500], Loss: 0.4169
Epoch [120/1500], Loss: 0.4106
Epoch [130/1500], Loss: 0.4046
Epoch [140/1500], Loss: 0.3987
Epoch [150/1500], Loss: 0.3930
Epoch [160/1500], Loss: 0.3874
Epoch [170/1500], Loss: 0.3819
Epoch [180/1500], Loss: 0.3764
Epoch [190/1500], Loss: 0.3707
Epoch [200/1500], Loss: 0.3649
Epoch [210/1500], Loss: 0.3590
Epoch [220/1500], Loss: 0.3529
Epoch [230/1500], Loss: 0.3466
Epoch [240/1500], Loss: 0.3401
Epoch [250/1500], Loss: 0.3333
Epoch [260/1500], Loss: 0.3261
Epoch [270/1500], Loss: 0.3184
Epoch [280/1500], Loss: 0.3105
Epoch [290/1500], Loss: 0.3024
Epoch [300/1500], Loss: 0.2943
Epoch [310/1500], Loss: 0.2859
Epoch [320/1500], Loss: 0.2771
Epoch [330/1500],

In [ ]:
# Evaluate the model on the test set
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation for inference
    predictions = model(X_test)
    predicted_labels = (predictions > 0.5).float()  # Convert outputs to 0 or 1 using 0.5 threshold

    # Calculate accuracy
    accuracy = accuracy_score(y_test, predicted_labels)
    print(f"Model accuracy on test set: {accuracy * 100:.2f}%")


Model accuracy on test set: 66.88%


## **Multulayer Perceptron**

In [ ]:
columns = X.columns

In [ ]:
def create_dataloader(dset, batch_size):
  X_train = dset[columns]

  if torch.cuda.is_available():
    X = torch.tensor(X_train.values, dtype=torch.float32).cuda()
    y = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1).cuda()
  else:
    X = torch.tensor(X_train.values, dtype=torch.float32)
    y = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)

  return DataLoader(list(zip(X, y)), shuffle=True, batch_size=batch_size)

train_loader = create_dataloader(X_train, 64)
test_loader = create_dataloader(X_test, len(X_test))

In [ ]:
class MultiLayerNet(torch.nn.Module):
    def __init__(self, input_size, frst_hidden_neurons, scnd_hidden_neurons, output_size):
        super(MultiLayerNet, self).__init__()
        self.first_hidden_layer = torch.nn.Linear(input_size, frst_hidden_neurons)
        self.second_hidden_layer = torch.nn.Linear(frst_hidden_neurons, scnd_hidden_neurons)

        self.output_layer = torch.nn.Linear(scnd_hidden_neurons, output_size)

    def forward(self, x):
        frst_hidden_output = torch.relu(self.first_hidden_layer(x))
        scnd_hidden_output = torch.softmax(self.second_hidden_layer(frst_hidden_output), dim=1)

        y_pred = torch.sigmoid(self.output_layer(scnd_hidden_output))

        return y_pred

In [ ]:
learningRate = 0.1
epochs = 1000

model = MultiLayerNet(8, 16, 16, 1)
if torch.cuda.is_available():
    model.cuda()

criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

In [ ]:
training_loss = []
eval_loss = []
acc_list = []

for epoch in range(epochs):
    for X_batch, y_batch in train_loader:
      optimizer.zero_grad()
      outputs = model(X_batch)
      loss = criterion(outputs, y_batch)
      loss.backward()

      # update parameters
      optimizer.step()

    if epoch % 100 == 0:
      print('epoch {}, loss {}'.format(epoch, loss.item()))

    training_loss.append(loss.item())


    # Validation
    for x__val_batch, y__val_batch in test_loader:
      # model.eval()
      val_outputs = model(x__val_batch)
      val_loss = criterion(val_outputs, y__val_batch)

      acc = (val_outputs.round() == y__val_batch).float().mean()

    if epoch % 100 == 0:
      print("Model accuracy: %.2f%%" % (float(acc)*100))
      print('epoch {}, val_loss {}\n'.format(epoch, val_loss.item()))


    eval_loss.append(val_loss.item())
    acc_list.append(float(acc))

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(training_loss, linestyle='-', color='b', label='Training Loss')
plt.plot(eval_loss, linestyle=':', color='r', label='Validation Loss')
plt.plot(acc_list, linestyle='-', color='g', label='Accuracy')

plt.xlabel('Epochs')
plt.ylabel('Cross Entropy Loss')
plt.title('Training Progress')

plt.grid(True)

# Show legend
plt.legend()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class AdvancedMultiLayerNet(nn.Module):
    def __init__(self, input_size, hidden_layer_sizes=[256, 128, 64, 32], output_size=1, dropout_prob=0.4, weight_decay=1e-5):
        super(AdvancedMultiLayerNet, self).__init__()

        # Dynamically create hidden layers
        self.hidden_layers = nn.ModuleList()
        input_channels = input_size
        
        # Create hidden layers based on the hidden_layer_sizes list
        for size in hidden_layer_sizes:
            self.hidden_layers.append(nn.Linear(input_channels, size))
            input_channels = size

        # Output layer
        self.output_layer = nn.Linear(input_channels, output_size)

        # Batch Normalization and Dropout layers
        self.batch_norms = nn.ModuleList()
        self.dropouts = nn.ModuleList()
        
        for size in hidden_layer_sizes:
            self.batch_norms.append(nn.BatchNorm1d(size))
            self.dropouts.append(nn.Dropout(dropout_prob))

    def forward(self, x):
        # Apply layers with ReLU, BatchNorm, Dropout, and LeakyReLU
        for i, hidden_layer in enumerate(self.hidden_layers):
            x = hidden_layer(x)
            x = self.batch_norms[i](x)  # Apply Batch Normalization
            x = F.leaky_relu(x, negative_slope=0.01)  # Use LeakyReLU instead of ReLU
            x = self.dropouts[i](x)  # Apply Dropout
        
        # Output layer with sigmoid activation for binary classification
        y_pred = torch.sigmoid(self.output_layer(x))
        return y_pred

epochs = 3000
learningRate = 0.01

model = AdvancedMultiLayerNet(input_size=8, hidden_layer_sizes=[256, 128, 64, 32])

criterion = torch.nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

if torch.cuda.is_available():
    model.cuda()